# GyroAI-SAT (v1)

Notebook simplificado para análise de orientação e Gimbal Lock usando dados TLE.

# Pré-processamento

Transformar os dados propagados em uma estrutura útil para IA.

In [8]:
import pandas as pd

# Exemplo: dataframe de orientação (Euler ou quaternions)
df = pd.DataFrame({
    'tempo': tempos,
    'q0': q0, 'q1': q1, 'q2': q2, 'q3': q3,
    'roll': roll, 'pitch': pitch, 'yaw': yaw
})

# Geração automática do rótulo (para treinar)
def rotular_gimbal_lock(pitch, tolerancia=2):
    if abs(pitch - 90) < tolerancia:
        return 'gimbal_lock'
    elif abs(pitch - 90) < tolerancia + 5:
        return 'alerta'
    else:
        return 'ok'

df['status'] = df['pitch'].apply(rotular_gimbal_lock)


NameError: name 'tempos' is not defined

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def criar_janelas(df, janela=10):
    X, y = [], []
    for i in range(len(df) - janela):
        X.append(df[['q0','q1','q2','q3']].iloc[i:i+janela].values)
        y.append(df['status'].iloc[i+janela])
    return np.array(X), np.array(y)

X, y = criar_janelas(df)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, shuffle=True)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))


In [ ]:
entrada = df[['q0','q1','q2','q3']].iloc[-10:].values.reshape(1, 10, 4)
status_predito = le.inverse_transform([np.argmax(model.predict(entrada))])[0]
print("STATUS PREVISTO:", status_predito)
